# Space Time Scan

# Using SatScan

- Start SaTScan and make a new session
- Under the "Input" tab:
    - Set the "Case File" to "chicago.cas"
    - Set the "Coordinates Files" to "chicago.geo"
    - Set "Coordinates" to "Cartesian"
    - Set "Time Precision" to Day
    - Set the "Study Period" from 2011-03-01 to 2011-09-27 (or whatever)
- Under the "Analysis" tab:
    - Select "Propsective Analysis" -> "Space-Time"
    - Select "Probability Model" -> "Space-Time Permutation"
    - Select "Time Aggregation" -> "1 Day"
- Under the "Output" tab:
    - Select the "Main Results File" to whatever
    
*Optionally* change the spatial and temporal window:

- Under "Analysis", click "Advanced":
    - Under "Spatial Window", select "is a circle with a ..."
    - Under "Temporal Window", select "Maximum Temporal Cluster Size" is ... days

In [1]:
%matplotlib inline
from common import *
datadir = os.path.join("//media", "disk", "Data")
south_side, points = load_data(datadir)